## Using the Gensim Model - Unsupervised.

In [7]:
!unzip /content/semi-supervised-lda.zip

Archive:  /content/semi-supervised-lda.zip
   creating: semi-supervised-lda/
  inflating: __MACOSX/._semi-supervised-lda  
  inflating: semi-supervised-lda/.DS_Store  
  inflating: __MACOSX/semi-supervised-lda/._.DS_Store  
   creating: semi-supervised-lda/data/
  inflating: __MACOSX/semi-supervised-lda/._data  
  inflating: semi-supervised-lda/data/.DS_Store  
  inflating: __MACOSX/semi-supervised-lda/data/._.DS_Store  
   creating: semi-supervised-lda/data/parsed_content/
  inflating: __MACOSX/semi-supervised-lda/data/._parsed_content  
   creating: semi-supervised-lda/data/raw_content/
  inflating: __MACOSX/semi-supervised-lda/data/._raw_content  
   creating: semi-supervised-lda/data/links/
  inflating: __MACOSX/semi-supervised-lda/data/._links  
  inflating: semi-supervised-lda/data/parsed_content/data_cleansed.txt  
  inflating: __MACOSX/semi-supervised-lda/data/parsed_content/._data_cleansed.txt  
  inflating: semi-supervised-lda/data/raw_content/regeneron-monoclonal-antibodies.

In [8]:
! pip3 install pyLDAvis

     |████████████████████████████████| 1.6MB 8.5MB/s 
     |████████████████████████████████| 552kB 45.8MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=8b11662c2b161d313d7ee5439cd142ffcf5a2ad80ae8830607082bc9207df152
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=40656e4dd12d55caca43d9f6fbaff90d0baa719c2fd6ef16c0b6403c37f3deb7
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [9]:
## import libraries.
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
## global vars.
data_cleansed_file="/content/semi-supervised-lda/data/parsed_content/data_cleansed.txt"

In [11]:
## Load cleansed data.
import pickle
data_cleansed = []
with open("/content/semi-supervised-lda/data/parsed_content/data_cleansed.txt", "rb") as fp:   # Unpickling
  data_cleansed = pickle.load(fp)

print(data_cleansed)

[['science', 'today', 'science', 'set', 'sight', 'mission', 'spacecraft', 'much', 'read', 'support', 'continue_reade', 'emirate', 'set', 'sight', 'delay', 'weather', 'mission', 'goal', 'make', 'contribution', 'government', 'really', 'hope', 'inspire', 'future', 'scientist', 'head', 'science', 'operation', 'manager', 'mission', 'update', 'girl', 'grow', 'look', 'book', 'neighbor', 'milky_way', 'describe', 'say', 'interview', 'realize', 'print', 'page', 'large', 'ever', 'see', 'planet', 'live', 'major', 'science', 'instead', 'aim', 'youth', 'science', 'technology', 'career', 'forge', 'career', 'pursue', 'heaven', 'year', 'old', 'head', 'science', 'operation', 'manager', 'space', 'probe', 'send', 'rocket', 'lift', 'spacecraft', 'call', 'hope', 'begin', 'bold', 'move', 'yet', 'country', 'look', 'establish', 'future', 'long', 'outlive', 'oil', 'wealth', 'see', 'space', 'program', 'way', 'accomplish', 'goal', 'much', 'news', 'next', 'month', 'month', 'line', 'allow', 'robotic', 'spacecraft',

In [12]:
# Create Dictionary
id2word = corpora.Dictionary(data_cleansed)

# Create Corpus
texts = data_cleansed

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [13]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('able', 1),
  ('accomplish', 1),
  ('ad_choice', 1),
  ('ahead', 2),
  ('aim', 2),
  ('airport', 1),
  ('allow', 1),
  ('almost', 2),
  ('also', 2),
  ('ambitious', 1),
  ('antenna', 3),
  ('appearance', 1),
  ('apprentice', 1),
  ('approach', 2),
  ('arrive', 1),
  ('ask', 1),
  ('assemble', 1),
  ('assignment', 1),
  ('astronomical', 1),
  ('atmosphere', 3),
  ('back', 1),
  ('bad', 1),
  ('ball', 1),
  ('battery', 2),
  ('begin', 2),
  ('big', 1),
  ('bold', 1),
  ('book', 1),
  ('build', 6),
  ('bunch', 1),
  ('buy', 3),
  ('ca', 1),
  ('calendar', 1),
  ('call', 1),
  ('camera', 4),
  ('capture', 1),
  ('car', 1),
  ('carbon', 1),
  ('career', 3),
  ('carefully', 1),
  ('cargo', 1),
  ('carry', 1),
  ('center', 1),
  ('challenge', 1),
  ('change', 2),
  ('charge', 2),
  ('chemical', 1),
  ('choice', 1),
  ('cit', 1),
  ('clear', 1),
  ('cloud', 1),
  ('collaborate', 1),
  ('college', 1),
  ('communicate', 2),
  ('company', 1),
  ('complete', 2),
  ('computer', 1),
  ('cone', 1)

In [14]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=7, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [15]:
## View Topics
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.014*"say" + 0.006*"mission" + 0.006*"science" + 0.006*"study" + '
  '0.005*"could" + 0.005*"may" + 0.005*"work" + 0.005*"school" + 0.005*"would" '
  '+ 0.005*"scientist"'),
 (1,
  '0.021*"say" + 0.016*"may" + 0.015*"virus" + 0.015*"people" + 0.010*"test" + '
  '0.009*"bar" + 0.008*"case" + 0.008*"infection" + 0.008*"study" + '
  '0.007*"time"'),
 (2,
  '0.008*"specie" + 0.007*"bear" + 0.005*"tree" + 0.005*"genetic" + '
  '0.005*"say" + 0.005*"lizard" + 0.005*"live" + 0.005*"gene" + '
  '0.004*"enslave" + 0.004*"find"'),
 (3,
  '0.046*"say" + 0.030*"state" + 0.014*"report" + 0.013*"bar" + 0.013*"case" + '
  '0.013*"datum" + 0.010*"may" + 0.010*"people" + 0.008*"help" + 0.008*"know"'),
 (4,
  '0.006*"antibiotic" + 0.003*"vibration" + 0.002*"seismic" + '
  '0.002*"seismometer" + 0.002*"drug_resistant" + 0.002*"quiet" + 0.002*"up" + '
  '0.002*"reduction" + 0.002*"earthquake" + 0.002*"ferret"'),
 (5,
  '0.028*"vaccine" + 0.023*"say" + 0.009*"company" + 0.007*"patient" + '
  '0.00

In [16]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score -- higher the better.
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_cleansed, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for


Perplexity:  -7.620903888354863

Coherence Score:  0.3773846924761238


In [17]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.125705  0.025295       1        1  36.789711
0     -0.016738  0.163368       2        1  32.228912
5      0.059444  0.003152       3        1  19.629528
3      0.360879 -0.065011       4        1   5.785297
2     -0.166772 -0.020113       5        1   3.495485
6     -0.186589 -0.066075       6        1   1.651223
4     -0.175929 -0.040615       7        1   0.419849, topic_info=          Term         Freq        Total Category  logprob  loglift
276        say  1366.000000  1366.000000  Default  30.0000  30.0000
652    vaccine   402.000000   402.000000  Default  29.0000  29.0000
629      state   167.000000   167.000000  Default  28.0000  28.0000
959        bar   329.000000   329.000000  Default  27.0000  27.0000
413       case   312.000000   312.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
2196      drug     0.445743    76.830032   Topic7  -6.4734   0.3234
321       team     0.320027    80.041893   Topic7  -6.8048  -0.0489
850   question     0.304711    73.850998   Topic7  -6.8538  -0.0174
133      human     0.285947   128.784286   Topic7  -6.9174  -0.6371
368   activity     0.255369    15.346703   Topic7  -7.0304   1.3771

[468 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
1421      4  0.799372        absent
6296      5  0.823406  accidentally
930       1  0.726244      accurate
930       3  0.155624      accurate
930       6  0.103749      accurate
...     ...       ...           ...
361       6  0.003225         would
363       1  0.164525          year
363       2  0.493575          year
363       3  0.314948          year
363       5  0.028204          year

[824 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 6, 4, 3, 7, 5])

## Using Correx for guided LDA

In [4]:
!pip3 install corextopic

In [5]:
import numpy as np
import scipy.sparse as ss
import matplotlib.pyplot as plt

from corextopic import corextopic as ct
from corextopic import vis_topic as vt # jupyter notebooks will complain matplotlib is being loaded twice

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer

%matplotlib inline

In [9]:
import pandas as pd
# Transform data into a sparse matrix
raw_file = "/content/raw_content_df.txt"
df = pd.read_csv(raw_file, sep=":")

vectorizer = CountVectorizer(stop_words='english', max_features=20000, binary=True)
doc_word = vectorizer.fit_transform(df.content)
doc_word = ss.csr_matrix(doc_word)



In [10]:
print(doc_word.shape)

(101, 12432)


In [12]:
words = list(np.asarray(vectorizer.get_feature_names()))

In [13]:
not_digit_inds = [ind for ind,word in enumerate(words) if not word.isdigit()]
doc_word = doc_word[:,not_digit_inds]
words    = [word for ind,word in enumerate(words) if not word.isdigit()]

doc_word.shape # n_docs x m_words

(101, 12185)

In [16]:
# Train the CorEx topic model with 50 topics
topic_model = ct.Corex(n_hidden=10, words=words, max_iter=200, verbose=False, seed=1)
topic_model.fit(doc_word, words=words);

In [17]:
# Print all topics from the CorEx topic model
topics = topic_model.get_topics()
for n,topic in enumerate(topics):
    topic_words,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

0: 07t11,rouge,bout,hear,sicker,alcohol,owners,injury,soaring,gov
1: miss,astronomical,eclipse,calendar,meteor,sync,x99ve,winter,event,experiencing
2: launch,scheduled,preview,artboard,ai,ai2html,v0,scoop,mission,artboard_1_copy_2
3: trying,decide,names,wall,senate,periods,lungs,uncertain,examples,1990s
4: attention,street,vice,warned,clinical,viral,talk,month,menu,wind
5: dr,indoor,chang,preparedness,indoors,maryland,kenneth,miles,expert,organization
6: additional,billion,press,operation,expects,gathered,doctors,associated,experimental,influence
7: wasn,question,details,watch,parents,x9cone,waves,modern,supplies,big
8: efforts,tracing,personnel,charge,doing,diverse,strongly,completed,setting,interview
9: touched,places,change,pre,apply,zone,stretch,protects,reuters,answers


In [19]:
# Anchor 'health', 'virus', 'corona', 'doctor' to first topic, 'treat', 'vaccine', 'clinic' to second topic, so on...
anchor_words = [['health', 'virus', 'corona', 'doctor'], ['treat', 'vaccine', 'clinic']]

anchored_topic_model = ct.Corex(n_hidden=10, seed=2)
anchored_topic_model.fit(doc_word, words=words, anchors=anchor_words, anchor_strength=6);

In [20]:
for n in range(len(anchor_words)):
    topic_words,_ = zip(*anchored_topic_model.get_topics(topic=n))
    print('{}: '.format(n) + ','.join(topic_words))

0: virus,health,positive,organizations,teachers,adults,exposed,classrooms,tracker,checker
1: vaccine,little,file,generated,need,million,infectious,treat,disease,planet
